In [1]:

import configparser
config = configparser.ConfigParser()
config.read('/Users/makbulhussain/Downloads/dwh.cfg')
print("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))

host=redshift.amazonaws.com dbname=dwh user=dwhuser password=Passw0rd port=5439


## Data wrangling

In [1]:
import pyspark
if 'sc' in locals():
    sc.stop()


In [22]:
# sc = pyspark.SparkContext(appName="maps_and_lazy_evaluation_example")

In [24]:
# from pyspark import SparkContext

# sc = SparkContext("local[*]", "my_spark_app")


In [2]:
log_of_songs = [
        "Despacito",
        "Nice for what",
        "No tears left to cry",
        "Despacito",
        "Havana",
        "In my feelings",
        "Nice for what",
        "despacito",
        "All the stars"
]

# parallelize the log_of_songs to use with Spark


# show the original input data is preserved


# create a python function to convert strings to lowercase
def convert_song_to_lowercase(song):
    return song.lower()

print(convert_song_to_lowercase("Songtitle"))


songtitle


In [3]:
from pyspark.sql import SparkSession

# Because we aren't running on a spark cluster, the session is just for development
spark = SparkSession \
    .builder \
    .appName("Maps and Lazy Evaluation Example 1") \
    .getOrCreate()


# Starting off with a regular python list
log_of_songs = [
        "Despacito",
        "Nice for what",
        "No tears left to cry",
        "Despacito",
        "Havana",
        "In my feelings",
        "Nice for what",
        "despacito",
        "All the stars"
]

# parallelize the log_of_songs to use with Spark
# distributed_song_log_rdd is an RDD (Reslient Distributed Dataset)
distributed_song_log_rdd = spark.sparkContext.parallelize(log_of_songs)

# notice we DO NOT use the .collect() method. What is the difference between
# .collect() and .foreach() ? 
# .collect() forces all the data from the entire RDD on all nodes 
# to be collected from ALL the nodes, which kills productivity, and could crash
# .foreach() allows the data to stay on each of the independent nodes

# show the original input data is preserved
distributed_song_log_rdd.foreach(print)

def convert_song_to_lowercase(song):
    return song.lower()

print(convert_song_to_lowercase("Havana"))


# toDF() Converts from an RDD to a DataFrame- this allows us to use convenient functions like show() 
lower_case_songs=distributed_song_log_rdd.map(convert_song_to_lowercase)
lower_case_songs.foreach(print)

# Show the original input data is still mixed case
distributed_song_log_rdd.foreach(print)

# Use lambda functions instead of named functions to do the same map operation
distributed_song_log_rdd.map(lambda song: song.lower()).foreach(print)
distributed_song_log_rdd.map(lambda x: x.lower()).foreach(print)

23/08/09 09:41:53 WARN Utils: Your hostname, Makbuls-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.10.30.219 instead (on interface en0)
23/08/09 09:41:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/09 09:41:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Nice for what                                                       (0 + 8) / 8]
despacito
All the stars
Havana
Nice for what
No tears left to cry
In my feelings
Despacito
Despacito
despacito                                                                       
all the stars
despacito
despacito
havana


havana


no tears left to cry
in my feelings
nice for what
nice for what
Despacito
Nice for what
No tears left to cry
In my feelings
despacito
All the stars
Havana
Nice for what
Despacito
despacito
all the stars
nice for what
in my feelings
despacito
nice for what
havana
no tears left to cry
despacito
nice for what
despacito
no tears left to cry
in my feelings
despacito
all the stars
nice for what
havana
despacito


In [4]:
# After some initialization, we'll convert the log of songs (just a normal Python list) 
# to a distributed dataset that Spark can use. This uses a special spark.sparkContext object. 
# The Spark Context has a method parallelize that takes a Python object and distributes the object across the machines in your cluster so Spark can process the dataset.


distributed_song_log_rdd = spark.sparkContext.parallelize(log_of_songs)


In [6]:
distributed_song_log_rdd.map(convert_song_to_lowercase)


PythonRDD[7] at RDD at PythonRDD.scala:53

In [5]:
# If we want to force Spark to take some action on the data, 
# we can use the collect function, which gathers the results from all of the machines in our cluster.
distributed_song_log_rdd.map(convert_song_to_lowercase).collect()



['despacito',
 'nice for what',
 'no tears left to cry',
 'despacito',
 'havana',
 'in my feelings',
 'nice for what',
 'despacito',
 'all the stars']

## Data Inputs and Outputs

In [8]:
from pyspark.sql import SparkSession


# Because we aren't running on a spark cluster, the session is just for development
spark = SparkSession \
    .builder \
    .appName("Our first Python Spark SQL example") \
    .getOrCreate()
# This should print the default configuration
print(
    spark.sparkContext.getConf().getAll()
)    


[('spark.driver.host', '10.10.30.219'), ('spark.app.startTime', '1691554314282'), ('spark.app.name', 'Maps and Lazy Evaluation Example 1'), ('spark.executor.id', 'driver'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethod

In [23]:
# This path resides on your computer or workspace, not in HDFS
path = "/Users/makbulhussain/Downloads/sparkify_log_small.json"
user_log_df = spark.read.json(path)

# See how Spark inferred the schema from the JSON file
user_log_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [24]:
print(
    user_log_df.describe()
)

DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]


In [16]:
user_log_df.show(n=1)

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------

In [25]:
print(
    user_log_df.take(1)
)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]


In [18]:
# We are changing file formats
out_path = "/Users/makbulhussain/Downloads/sparkify_log_small.csv"



# The filename alone didn't tell Spark the actual format, we need to do it here
user_log_df.write.mode("overwrite").save(out_path, format="csv", header=True)

# Notice we have created another dataframe here
# We wouldn't usually read the data that we just wrote
# This does show, however, that the read method works with
# Different data types
user_log_2_df = spark.read.csv(out_path, header=True)
user_log_2_df.printSchema()


root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: string (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: string (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: string (nullable = true)
 |-- sessionId: string (nullable = true)
 |-- song: string (nullable = true)
 |-- status: string (nullable = true)
 |-- ts: string (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [19]:
# Choose two records from the CSV file
print(
    user_log_2_df.take(2)
)    


[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession='112', lastName='Matthews', length='232.93342', level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration='1509380319284', sessionId='5132', song='Christmas Tears Will Fall', status='200', ts='1513720872284', userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'), Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession='7', lastName='Chase', length='195.23873', level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration='1512718541284', sessionId='5027', song='Cheryl Tweedy', status='200', ts='1513720878284', userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000')]


In [22]:
# Show the userID column for the first several rows
user_log_2_df.select("userID").show(3)


+------+
|userID|
+------+
|  1046|
|  1000|
|  2219|
+------+
only showing top 3 rows



In [21]:
# 
print(
user_log_2_df.take(1)
)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession='112', lastName='Matthews', length='232.93342', level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration='1509380319284', sessionId='5132', song='Christmas Tears Will Fall', status='200', ts='1513720872284', userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]


In [36]:
# Simple Rename
user_log_2_df.withColumnRenamed('artist','superartist').show(1)

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|  superartist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------

In [37]:
# Double the length
user_log_2_df.withColumn('doublelength',user_log_2_df['length']*2).show(1)


+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+------------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|doublelength|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+------------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|   465.86684|
+-------------+---------+---------+------+-------------+--------+---------+-----+---------------

## Using SQL

In [27]:
# Register the DataFrame as a SQL temporary view
user_log_2_df.createOrReplaceTempView("people")


In [28]:
sql_results = spark.sql("SELECT * FROM people")


In [29]:
sql_results


DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]

In [30]:
sql_results.show(1)


+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------

In [34]:
spark.sql("SELECT * FROM people WHERE artist='Showaddywaddy'").show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------